In [1]:
import sys
import os
sys.path.insert(0, './confined-colloids')

import lammps2d as lmp
import numpy as np
import matplotlib.pyplot as plt

import getpass
import pysftp

%reload_ext autoreload
%autoreload 2

In [2]:
def run_sim_and_send(total_time,password):
    
    print(total_time)
    
    angle = 15
    radius = 1.4
    max_freq = 20

    region, initial_positions = lmp.initial_setup(n_of_particles = 2000, packing=0.175, height = 4.0)

    sim = lmp.sim(initial_positions,
              magnitude = 7.28,
              file_name = "dynamic_angle15_0_20Hz_"+str(total_time)+"sec_large",
              dir_name = "/home/aortiza/Rupture/",
              radius = radius,
              diffusion = 0.073,
              frequency=str(max_freq/total_time*1e-6)+"*time",
              susceptibility = 0.4,
              angle=angle,
              framerate=1e2,
              timestep=0.5e-4,
              total_time = total_time,
              temperature = 300,
              stamp_time = False,
              space = {'region':region,'boundary':['p','p','f'],'walls':[False,False,True]})

    sim.generate_scripts()
    sim.run()
    
    lmp.lammpstrj_to_hdf5(os.path.join(sim.sim_parameters.dir_name,sim.sim_parameters.file_name))
    
    directory = "Antonio/SimulationsOfDimerSystem/RuptureTransition/"
    
    with pysftp.Connection(host="161.116.81.190", username="aortiza", password=password) as magnetic:

        if not magnetic.isdir(directory):
            magnetic.makedirs(directory)

        with magnetic.cd(directory):
            magnetic.put(sim.base_name+'.lammpstrj')
            magnetic.put(sim.base_name+'.jp')
            magnetic.put(sim.base_name+'.lmpin')
            magnetic.put(sim.base_name+'.hd5')
        
    os.remove(sim.base_name+'.lammpstrj')
    os.remove(sim.base_name+'.jp')
    os.remove(sim.base_name+'.lmpin')
    os.remove(sim.base_name+'.hd5')

In [3]:
def run_several(password):
    for time in np.round(np.logspace(np.log(300)/np.log(10),np.log(102.48537891)/np.log(10),7)):
        run_sim_and_send(time,password)

In [ ]:
run_several(getpass.getpass())

········
300.0


HBox(children=(IntProgress(value=0, max=30001), HTML(value='')))